In [ ]:
import numpy as np
import pandas as pd

train_df = pd.read_csv('../house_data/train.csv')
test_df = pd.read_csv('../house_data/test.csv')

# Get target variable
y_train = train_df['price'].values / 1000
y_test = test_df['price'].values / 1000

print("="*70)
print("PROBLEM 4: POLYNOMIAL REGRESSION")
print("="*70)

# ============================================
# HELPER FUNCTIONS
# ============================================

def train_linear_regression(X, y):
    """
    Train linear regression using closed-form solution
    theta = (X^T X)^(-1) X^T y
    """
    XTX = X.T @ X
    theta = np.linalg.inv(XTX) @ X.T @ y
    return theta


def predict(X, theta):
    """Predict response for new data points"""
    return X @ theta


def compute_mse(y_true, y_pred):
    """Compute Mean Squared Error"""
    return np.mean((y_true - y_pred) ** 2)


def compute_r2(y_true, y_pred):
    """Compute R-squared score"""
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)


def create_polynomial_features(X, degree):
    """
    Create polynomial features up to given degree
    
    Example: If X = [2, 3] and degree = 3
    Returns: [[2, 4, 8],    # [2^1, 2^2, 2^3]
              [3, 9, 27]]   # [3^1, 3^2, 3^3]
    
    Parameters:
    -----------
    X : array (n_samples,)
        Single feature values
    degree : int
        Maximum degree of polynomial
    
    Returns:
    --------
    X_poly : array (n_samples, degree)
        Polynomial features [X, X^2, X^3, ..., X^degree]
    """
    n_samples = len(X)
    X_poly = np.zeros((n_samples, degree))
    
    # Create features: X, X^2, X^3, ..., X^p
    for d in range(1, degree + 1):
        X_poly[:, d-1] = X ** d
    
    return X_poly


# ============================================
# STEP 1: EXTRACT AND PREPARE DATA
# ============================================
print("\nSTEP 1: Extract sqft_living feature")
print("-" * 70)

X_train_sqft = train_df['sqft_living'].values
X_test_sqft = test_df['sqft_living'].values

print(f"Train samples: {len(X_train_sqft)}")
print(f"Test samples:  {len(X_test_sqft)}")
print(f"sqft_living range: [{X_train_sqft.min():.0f}, {X_train_sqft.max():.0f}]")


# ============================================
# STEP 2: STANDARDIZE FEATURE
# ============================================
print("\nSTEP 2: Standardize sqft_living")
print("-" * 70)

mean_sqft = X_train_sqft.mean()
std_sqft = X_train_sqft.std(ddof=1)

X_train_sqft_scaled = (X_train_sqft - mean_sqft) / std_sqft
X_test_sqft_scaled = (X_test_sqft - mean_sqft) / std_sqft

print(f"Original - Mean: {mean_sqft:.2f}, Std: {std_sqft:.2f}")
print(f"Scaled   - Mean: {X_train_sqft_scaled.mean():.6f}, Std: {X_train_sqft_scaled.std(ddof=1):.6f}")


# ============================================
# STEP 3: TRAIN POLYNOMIAL MODELS
# ============================================
print("\nSTEP 3: Train polynomial models for degrees 1-5")
print("-" * 70)

degrees = [1, 2, 3, 4, 5]
results = []

for p in degrees:
    print(f"\n{'='*70}")
    print(f"DEGREE p = {p}")
    print('='*70)
    
    # Create polynomial features: [X, X^2, X^3, ..., X^p]
    X_train_poly = create_polynomial_features(X_train_sqft_scaled, p)
    X_test_poly = create_polynomial_features(X_test_sqft_scaled, p)
    
    print(f"Polynomial feature matrix shape: {X_train_poly.shape}")
    print(f"Features: ", end="")
    for d in range(1, p+1):
        print(f"X^{d}", end=" " if d < p else "\n")
    
    # Add intercept column: [1, X, X^2, X^3, ..., X^p]
    X_train_poly_int = np.c_[np.ones(X_train_poly.shape[0]), X_train_poly]
    X_test_poly_int = np.c_[np.ones(X_test_poly.shape[0]), X_test_poly]
    
    print(f"With intercept: {X_train_poly_int.shape}")
    
    # Train model using closed-form solution
    theta_poly = train_linear_regression(X_train_poly_int, y_train)
    
    # Display coefficients
    print(f"\nCoefficients:")
    print(f"  theta_0 (intercept) = {theta_poly[0]:.4f}")
    for d in range(1, min(p+1, 4)):  # Show first 3 coefficients
        print(f"  theta_{d} (X^{d})       = {theta_poly[d]:.4f}")
    if p > 3:
        print(f"  ... (and {p-3} more coefficients)")
    
    # Make predictions
    y_train_pred_poly = predict(X_train_poly_int, theta_poly)
    y_test_pred_poly = predict(X_test_poly_int, theta_poly)
    
    # Compute metrics
    train_mse_poly = compute_mse(y_train, y_train_pred_poly)
    train_r2_poly = compute_r2(y_train, y_train_pred_poly)
    test_mse_poly = compute_mse(y_test, y_test_pred_poly)
    test_r2_poly = compute_r2(y_test, y_test_pred_poly)
    
    print(f"\nPerformance Metrics:")
    print(f"  Training - MSE: {train_mse_poly:>12,.2f}  |  R2: {train_r2_poly:>7.4f}")
    print(f"  Testing  - MSE: {test_mse_poly:>12,.2f}  |  R2: {test_r2_poly:>7.4f}")
    
    # Store results
    results.append({
        'Degree': p,
        'Train_MSE': train_mse_poly,
        'Train_R2': train_r2_poly,
        'Test_MSE': test_mse_poly,
        'Test_R2': test_r2_poly
    })


# ============================================
# STEP 4: SUMMARY AND ANALYSIS
# ============================================
print("\n" + "="*70)
print("SUMMARY TABLE")
print("="*70)

results_df = pd.DataFrame(results)
print("\n" + results_df.to_string(index=False))

# Find best models
best_train_idx = results_df['Train_MSE'].idxmin()
best_test_idx = results_df['Test_MSE'].idxmin()

print(f"\n{'='*70}")
print("ANALYSIS")
print('='*70)
print(f"\nBest Training Performance: Degree {results_df.loc[best_train_idx, 'Degree']} "
      f"(MSE: {results_df.loc[best_train_idx, 'Train_MSE']:.2f})")
print(f"Best Testing Performance:  Degree {results_df.loc[best_test_idx, 'Degree']} "
      f"(MSE: {results_df.loc[best_test_idx, 'Test_MSE']:.2f})")

# Check for overfitting
if best_train_idx != best_test_idx:
    print(f"\n[!] OVERFITTING DETECTED:")
    print(f"   Degree {results_df.loc[best_train_idx, 'Degree']} fits training data best")
    print(f"   But degree {results_df.loc[best_test_idx, 'Degree']} generalizes better to test data")

PROBLEM 4: POLYNOMIAL REGRESSION

STEP 1: Extract sqft_living feature
----------------------------------------------------------------------
Train samples: 1000
Test samples:  1000
sqft_living range: [380, 6070]

STEP 2: Standardize sqft_living
----------------------------------------------------------------------
Original - Mean: 2051.20, Std: 887.93
Scaled   - Mean: 0.000000, Std: 1.000000

STEP 3: Train polynomial models for degrees 1-5
----------------------------------------------------------------------

DEGREE p = 1
Polynomial feature matrix shape: (1000, 1)
Features: X^1
With intercept: (1000, 2)

Coefficients:
  θ₀ (intercept) = 520.4148
  θ_1 (X^1)       = 239.2632

Performance Metrics:
  Training - MSE:    57,947.53  |  R²:  0.4967
  Testing  - MSE:    88,575.98  |  R²:  0.4687

DEGREE p = 2
Polynomial feature matrix shape: (1000, 2)
Features: X^1 X^2
With intercept: (1000, 3)

Coefficients:
  θ₀ (intercept) = 484.7817
  θ_1 (X^1)       = 194.8233
  θ_2 (X^2)       = 35.6688

Observations:

Training Performance: As polynomial degree increases from 1 to 5, training MSE consistently decreases (57,948 -> 52,626) and R^2 increases (0.497 -> 0.543). This indicates the model is fitting the training data progressively better with more complex polynomials.
Testing Performance: Test MSE is lowest at degree 2 (71,792) with R^2 = 0.569. Beyond degree 2, test MSE increases dramatically, reaching 570,617 at degree 5, with negative R^2 values at degrees 4 and 5, indicating performance worse than a horizontal line.
Overfitting: Clear overfitting occurs at degrees 3-5. While these models fit training data better, they perform catastrophically on test data. The negative R^2 values mean the model is making predictions worse than simply predicting the mean price. High-degree polynomials capture noise and random fluctuations in training data rather than true underlying patterns.
Optimal Degree: Degree 2 (quadratic) provides the best balance, achieving the lowest test MSE and highest test R^2 (0.569). This suggests the relationship between square footage and price follows a quadratic curve rather than a linear or higher-order relationship.

Conclusion:
The optimal polynomial degree is p = 2. This demonstrates the bias-variance tradeoff: low degrees (p=1) underfit with high bias, while high degrees (p>=3) overfit with high variance. Degree 2 strikes the right balance for generalization to unseen data.